#  Exploration des articles de 1920 (thème : immigration de la prese belge)

Dans cette étape, j’explore le sous-ensemble 1920 du corpus CAMille pour repérer un article pertinent sur l’immigration. Je vérifie le chemin, je compte les fichiers, j’applique un filtre par mots-clés, puis je regarde des extraits avant de choisir un article et d’enregistrer sa référence.

## 1) Imports et chargement du sous-corpus

Je charge les bibliothèques nécessaires et je récupère la liste des fichiers .txt du dossier data/txt. Je vérifie que je suis bien entre 500 et 999 documents comme demandé.

In [2]:
import spacy

nlp = spacy.load("fr_core_news_sm")
print("Modèle spaCy chargé")

Modèle spaCy chargé


In [3]:
import os, re
from collections import Counter

data_path = "./data/txt"  
files = sorted([f for f in os.listdir(data_path) if f.endswith(".txt")])

print("Nombre de fichiers :", len(files))
print("Exemples :", files[:5])

assert 500 <= len(files) <= 999, "Le sous-corpus doit contenir entre 500 et 999 documents."

Nombre de fichiers : 999
Exemples : ['KB_JB421_1920-01-07_01-00001.txt', 'KB_JB421_1920-01-07_01-00002.txt', 'KB_JB421_1920-01-07_01-00003.txt', 'KB_JB421_1920-01-07_01-00004.txt', 'KB_JB421_1920-03-03_01-00001.txt']


## 2) Construction  du  texte d’échantillon 

Pour éviter de tout traiter d’un coup, je construis un échantillon de 30 fichiers (c’est suffisant pour tester NER + mots-clés). Je concatène leur contenu dans une seule variable sample_text.

In [4]:
sample_files = files[:30]

sample_text = ""
for f in sample_files:
    with open(os.path.join(data_path, f), encoding="utf-8", errors="replace") as fh:
        sample_text += fh.read() + "\n"

print("Taille échantillon (caractères) :", len(sample_text))
print(sample_text[:800])

Taille échantillon (caractères) : 1032828
24' ANNEE, N* S. Le N* 15 centimes. Vendredi 77777g 7 .Janvier 1920.. ■5/» L'AVENIR DU PUBLICITÉ : «WKON'CBS : la. ligne. 1 fr.; RECLAMES : la ligne, 2 fr.; FAITS DIVERS, AVIS JUDICIAIRES, * FINANCIERS, NECROLOGIQUES ; la ligne, 2, 3 et 4 fr. — On traite à forfait pour les : annonc-- périodiques. — Les annonces pour la province de Luxembourg, les cantons limitrophes et le Grand-Duché sont reçues au bureau du lournal. Pour le reste du pays • et l'étranger, s'adresser exclusivement à l'administration de la Nation Belge, boulevard ! Anspach, 2-4, Bruxelles. QUOTIDIEN Rue de Luxembourg, 42, ARLON ABONNEMENTS : Belgique ; Un an, 34 francs; six mois, 17 fr. 50; trois mois 9 francs; un mois 3 francs. -- Etranger : le port en plus. On s'abonne à tous les bureaux de poste. -• Téléphone, Arlon 23 Co


## 3) Entités nommées (NER)

Je tente une reconnaissance d’entités nommées (personnes, lieux, organisations) avec spaCy. Cette étape me  permet d’identifier les acteurs et institutions qui reviennent dans le discours.

In [7]:
try:
    doc = nlp(sample_text)
    print("OK, doc créé ")
    print("Nb entités :", len(doc.ents))
except Exception as e:
    print("Erreur spaCy :", type(e).__name__, "-", e)

Erreur spaCy : ValueError - [E088] Text of length 1032828 exceeds maximum of 1000000. The parser and NER models require roughly 1GB of temporary memory per 100,000 characters in the input. This means long texts may cause memory allocation errors. If you're not using the parser or NER, it's probably safe to increase the `nlp.max_length` limit. The limit is in number of characters, so you can check whether your inputs are too long by checking `len(text)`.


## 4) Top entités (fréquences)

Je compte les entités les plus fréquentes pour repérer les noms qui structurent le corpus (pays, institutions, personnes).

In [8]:
ent_counter = Counter(ents)
for (text, label), c in ent_counter.most_common(20):
    print(f"{text} ({label}) — {c}")

NameError: name 'ents' is not defined

## 5) Extraction de mots-clés avec YAKE

J’extrais des mots-clés automatiquement sur le même échantillon. Cela donne des expressions “signifiantes” (souvent plus pertinentes que les simples fréquences).

In [ ]:
import yake

kw_extractor = yake.KeywordExtractor(lan="fr", top=20)
keywords = kw_extractor.extract_keywords(sample_text)

keywords[:20]

## 6)  Afficher uniquement les bigrammes

Pour améliorer l’interprétation, je garde uniquement les mots-clés en 2 mots (bigrams), souvent plus informatifs.

In [ ]:
bigrams = []
for kw, score in keywords:
    if len(kw.split()) == 2:
        bigrams.append((kw, score))

bigrams

In [ ]:
# Filtrer les articles 1920 liés à l'immigration (mots-clés simples) 
keywords = [
    "immigration", "immigrant", "immigrants",
    "étranger", "étrangère", "étrangers",
    "réfugié", "réfugiés", "asile", "naturalisation", "immigration", "immigrant", "immigrants", "étranger", "étrangère", "étrangers",
    "réfugié", "réfugiés", "asile", "naturalisation",
    "séjour", "permis de séjour", "titre de séjour", "carte d’identité d’étranger",
    "visa", "passeport", "laissez-passer", "frontière", "douanes", "contrôle",
    "expulsion", "refoulement", "internement",
    "administration des étrangers", "police des étrangers", "office des réfugiés",
    "consulat", "ministère de l’intérieur",
    "apatride", "demandeur d’asile",
    "rapatriement", "déplacés", "secours", "réinstallation",
    "main-d’œuvre étrangère", "recrutement", "logement", "ouvriers étrangers"
]

articles_retenus = []
for fname in files_1920:
    path = os.path.join(data_path, fname)
    with open(path, encoding="utf-8") as f:
        txt = f.read().lower()
    if any(k in txt for k in keywords):
        articles_retenus.append(fname)

print("Nombre d'articles potentiels :", len(articles_retenus))
articles_retenus[:10]